In [61]:
a = list(range(3))
print(a)
import numpy as np
print(len(a))

[0, 1, 2]
3


In [93]:

def test(k, g):
    for i in list(range(3)):
        df = {}
        dic = {}
        for i in g:
            for j in k:    
                dic1 = {'fpr': [], 'tpr': [], 'roc_auc': []}
                dic2 = {'precision': [], 'recall': [], 'f1': [], 'conf_matix': []}
                dic[j] = [dic1, dic2]
            df[i] = [dic]
            
    return df
    
g = ['a', 'b', 'c']
k = [1, 2, 3]
df= test(k, g)
print(df)
# print(df[g[0]][0][k[0]][0])
# print(dic[k[0]][0]['fpr'])

 

{'a': [{1: [{'fpr': [], 'tpr': [], 'roc_auc': []}, {'precision': [], 'recall': [], 'f1': [], 'conf_matix': []}], 2: [{'fpr': [], 'tpr': [], 'roc_auc': []}, {'precision': [], 'recall': [], 'f1': [], 'conf_matix': []}], 3: [{'fpr': [], 'tpr': [], 'roc_auc': []}, {'precision': [], 'recall': [], 'f1': [], 'conf_matix': []}]}], 'b': [{1: [{'fpr': [], 'tpr': [], 'roc_auc': []}, {'precision': [], 'recall': [], 'f1': [], 'conf_matix': []}], 2: [{'fpr': [], 'tpr': [], 'roc_auc': []}, {'precision': [], 'recall': [], 'f1': [], 'conf_matix': []}], 3: [{'fpr': [], 'tpr': [], 'roc_auc': []}, {'precision': [], 'recall': [], 'f1': [], 'conf_matix': []}]}], 'c': [{1: [{'fpr': [], 'tpr': [], 'roc_auc': []}, {'precision': [], 'recall': [], 'f1': [], 'conf_matix': []}], 2: [{'fpr': [], 'tpr': [], 'roc_auc': []}, {'precision': [], 'recall': [], 'f1': [], 'conf_matix': []}], 3: [{'fpr': [], 'tpr': [], 'roc_auc': []}, {'precision': [], 'recall': [], 'f1': [], 'conf_matix': []}]}]}
